# Presentation on SIMM and Athena SIMM Calculator

In [2]:
import pandas as pd
import CPG.SIMM.SIMMLocalCalculator.capital.non_cleared_im.simm.unittest.simm_calculators_local as sc

## SIMM Methodology

I'll briefly introduce the SIMM methodology, and go over some examples on how to calculate IM based on the SIMM methodology. The official SIMM methodology document can be found [here](http://www2.isda.org/search?headerSearch=1&keyword=simm)

### Brief Description of SIMM Methodology

* SIMM is a sensitivity based method for calculating initial margins. 
* Each Trade is assigned to one of the 4 product classes. SIMM is calculated seperately for each of them.
* Within each product class, initial margin is calculated by aggregating among the 6 risk classes
* For each risk class, initial margin is the sum of Delta Margin, Vega Margin, Curvature Margin and Base Correlation Margin.

* Product classes: RatesFX, Credit, Equity and Commodity
* Risk classes: Interest Rate, Credit (Qualifying), Credit (Non-Qualifying), Equity, Commodity and FX.



** Remark **

* A trade that is assigned to one of the product classes can have risk in multiple risk classes. SIMM is calculated per risk classes within that product class. 
* Example: Equity derivatives can have risk in both IR and equity risk class. 

### Delta Margin for IR Risk Class

* ** How do we calculate the Delta margin?**

Roughly speaking, we aggregate weighted sensitivities across different levels of buckets.

* ** What are the risk factors for IR risk class?**

We use $(k, i)$ to denote a risk factor, where $k$ is the rate tenor, and $i$ is the index name of the sub yield curve.

* ** What is a sub yield curve? **

Examples of sub yield curve: 'OIS_USD', 'Libor3m_USD', etc.

** In a summary, risk is seperately bucketed by currency, tenor and sub yiled curve index. i.e. a tuple (b, k, i).**

There are also inflation and xccy that should be taken into consideration.

Here is the methodology. 
* We found the net sensitivity $s^b_{k, i}$ for each currency $b$ to each risk factor $(k, i)$. Then we weight them by$$
WS^b_{ik} = rw^b_k\cdot s^b_{ik}\cdot CR_b
$$

Here $rw^b_k$ is the risk weight per tenor vertex $k$, and $CR^b$ is the concentration risk factor for each currency $b$, given by $$
CR_b = \max\biggl(1, \sqrt{\frac{\biggl|\sum_{k, i}s_{k, i}\biggr|}{T_b}}\biggr)
$$
where $T_b$ is the threshold for each currency $b$.

* Then we aggregate the weighted sensitivities according to curve index $(i, j)$ and tenor $(k, l)$ within each currency $b$ by$$
K^b = \sqrt{\sum_{i,j}\biggl(\sum_{k, l}WS^b_{jk}\rho_{kl}WS^b_{il}\biggr)\phi_{ij}}
$$
where $\rho_{kl}$ is the intra-bucket correlation, and $\phi_{ij} = 0.982$ is the universal correlation between two sub yiled curves. 

* Now we aggregate across buckets by$$
K = \sqrt{\sum_b(K^b)^2+\sum_{b, c}\gamma_{bc}g_{bc}S^bS^c}
$$
where $$
g_{bc} = \frac{\min(CR_b, CR_c)}{\max(CR_b, CR_c)}
$$
$$
S_b=\max(\min(\sum_{i, k}WS^b_{ik}, K_b), -K^b)
$$
and $\gamma_{bc}$ is the cross-bucket correlation. 

Here is a list of all important parameters we need:
* Risk weight $rw^b_k$ for each risk factor tenor $k$ and currency $b$;
* Concenration threshold $T_b$ for each currency $b$;
* Intra-bucket correlation $\rho_{kl}$ and yield curve correlation $\phi_{ij}$;
* Cross-bucket correlation $g_{bc}$.

### Delta Margin for Other Risk Classes

A few differences are applied for risk classes other than Interest Rates. In general, sensitivities should be firstly assigned to one of the buckets as specified in the ISDA SIMM Methodology document.

#### Risk classes?

* For Credit Qualifying/Non-Qualifying, risk factors are five credit spreads fro each issuer/seniority (Qualifying) or issuer/tranche (non-Qualifying) pair at vertices 1y, 2y, 3y, 5y and 10y. For each pair, at each vertex the sensitivities shall be netted across credit spreads to give the risk at that vertex.
* For Commodity, risk factors are all the commodity prices.
* For Equity, risk factors are all the equity prices.
* For FX, risk factors are all the exchange rates.

Methodology:
* $$
WS^b_k = rw^b_k\cdot s^b_k\cdot CR_k
$$
where

$$
CR^b_k = \max\biggl(1, \sqrt{\frac{\biggl|\sum_js_j\biggr|}{T_b}}\biggr)
$$
for credit risk, and summation is taken for all $j$ that have the same issuer and seniority with $k$.
$$
CR^b_k = \max\biggl(1, \sqrt{\frac{|s_k|}{T_b}}\biggr)
$$
for other risk classes.

* Aggregate intra-bucket
$$
K^b = \sqrt{WS^b_k\rho_{kl}f_{kl}WS^b_l}
$$
where$$
f_{kl} = \frac{\min(CR_k, CR_l)}{\max(CR_k. CR_l)}
$$

* Aggregate $K^b$ across buckets:$$
K = \sqrt{\sum_b(K^b)^2+\sum_{b,c}\gamma_{bc}S^bS^c}+K^{residual}
$$
where the only difference from the IR case is the additional residual part that comes from the residual bucket, denoting the class that lies outside of all the other defined buckets.

### Vega Margin for Interest Rate Risk Class

The calculation of vega margin is similar to the delta margin case, step by step as follows:

* Netting vega sensitivities across instruments, and weight by vega risk weights:$$
VR_k = vrw^b_k\biggl(\sum_{i}vr^b_{ik}\biggr)VCR_b
$$
where $VCR_b$ is the vega concentration risk $$
VCR_b = \max\biggl(1, \sqrt{\frac{\bigg|\sum_{ik}vr^b_{ik}\bigg|}{VT_b}}\biggr)
$$
with the threshold $VT_b$ 

* We now aggregate in an intra-bucket manner as follows:$$
K^b = \sqrt{VR\cdot\Sigma\cdot VR^T}
$$
where $\Sigma$ is the intra-bucket correlation matrix given by $(\rho_{kl}\cdot f_{kl})$. For IR risk class, $f_{kl}=1$.

* Now aggregate across buckets. $$
K = \sqrt{\sum_b(K^b)^2+\sum_{b, c}\gamma_{bc}g_{bc}S^bS^c} + K_{residual}
$$

### Vega Margin for Other Risk Classes

* Netting vega sensitivities across instruments, and weight by vega risk weights:$$
VR_k = vrw^b_k\biggl(\sum_{i}vr^b_{ik}\biggr)VCR_k
$$
where $VCR_k$ is the vega concentration risk $$
VCR_k = \max\biggl(1, \sqrt{\frac{\bigg|\sum_{i}vr^b_{ik}\bigg|}{VT_b}}\biggr)
$$
with the threshold $VT_b$ for Equity, FX and Commodity. 

For Credit, $VCR_k$ is given by$$
VCR_k = \max\biggl(1, \sqrt{\frac{\bigg|\sum_{ij}vr^b_{ij}\bigg|}{VT_b}}\biggr)
$$
where $j$ is taken over all tenors of the same issuer/seniority curve as the risk factor $k$.

* We now aggregate in an intra-bucket manner as follows:$$
K^b = \sqrt{VR\cdot\Sigma\cdot VR^T}
$$
where $\Sigma$ is the intra-bucket correlation matrix given by $(\rho_{kl}\cdot f_{kl})$. Here $f_{kl}$ is defined by$$
f_{kl} = \frac{\min(VCR_k, VCR_l)}{\max(VCR_k, VCR_l)}
$$

* Now aggregate across buckets. $$
K = \sqrt{\sum_b(K^b)^2+\sum_{b, c}\gamma_{bc}g_{bc}S^bS^c} + K_{residual}
$$

### Curvature Margin Calculation
Curvature margin is calculated based on vega margin. 
* $$
CVR^b_{ik} = SF(t_k)*vr^b_{ik}
$$
where $SF(t_k)$ is a function of $t_k$, which is a scaling factor of time to maturity corresponding to $t_k$. It only takes values on the standard tenor of 2W, 1M, ...,30Y.

* We then net $CVR^b_{ik}$ across instrument $i$ by$$
CVR^b_k = \sum_iCVR^b_{ik}
$$

* Now aggregate intra-bucket by$$
K^b = \sqrt{CVR^b_k\rho^2_{kl}CVR^b_l}
$$

* Now aggregate across bucket by$$
CM_{non-res} = \max\biggl(\sum_{b,k} CVR^b_k+\lambda\sqrt{\sum_b(K^b)^2+\sum_{b,c}\gamma^2_{bc}S_bS_c}, 0\biggr)
$$
where$$
\lambda = (\Phi^{-1}(99.5\%)^2-1)(1+\theta)-\theta
$$
and$$
\theta = \min\biggl(\frac{\sum_{b,k}CVR^b_k}{\sum_{b,k}|CVR^b_k|}, 0\biggr)
$$
$\gamma_{bc}$ and $S^b$ are defined as above.

We do the same calculation for the residual bucket as well, except that we do not need to aggregate across bucket in this case(single bucket).

* Curvature margin is finally equal to$$
CM = CM_{non-res}+CM_{residual}
$$

### Base Correlation Risk

Applied to the Credit (Qualifying) risk class.

The Base Correlation risk factor $k$ is the index family such as CDX IG.

1. $$
WS_k = rw_k\cdot s_k
$$

2. Base Correlation Margin is calculated via$$
BCM = \sqrt{WS_k\rho_{kl}WS_l}
$$

### Example: Manually Calculate IM of a Simple Hypothetical Trade

## Input of Athena SIMM Calculator

### How to access SIMM production run inputs

* Please follow the [instructions](https://confluence.uk.jpmorgan.com/confluence/display/RTDR/NCIM+Reports) on Accessing NCIM reports on the Gauss Download Console, which provides production calculator results and calculator input for each day.

* Choose **NCIM Calculator Inputs (gz)**.

* Download, extract and save the calculator input to a local path (As an example here I downloaded the calculator inout on 05/31/2017). 

In [4]:
input_path = r'C:\NonClearedIMSensPrep\Backtesting1d10d\AllInputs\Gauss_NCIM_Calculator_Inputs_Run5_171522819184_20170531\171522819184\calculator_inputs'

### Looking into the Inputs

Three sub-directories: `\model`, `\sens` and `\trade`:
* `\model` contains SIMM model parameters;
* `\sens` contains sensitivities that are put seperately for 6 different risk classes: Interest Rate, Credit (Qualifying and Non-Qualifying), Commodity, Equity and FX; as well as risk factor data.
* `\trade` contains trade data file `\trade\trade_data.csv`.

### How do we build up the SIMM model via the input folder?

Let's compare with our Delta margin model for IR risk class.

|Parameters used in Delta Margin for IR | Implemented|
|--------------- | -----------|
| Risk factor | `\sens\risk_factor_data_GIRR.csv` |
| Classification of low/regular/high vol currency | `\model\model_params_GIRR.csv` |
| Risk weight    | `\model\bucket_weight_GIRR.csv` |
| Risk concentration threshold | `\model\model_params_GIRR.csv` |
| Intra-bucket correlation | `\model\intra_bucket_correlation_GIRR.csv` |
| Yield curve correlation | `\model\model_params_GIRR.csv` |
| Cross bucket correlation | `\model\bucket_correlation_GIRR.csv` |

| Other information | Implemented |
| ----------------- | ----------- |
| FX rate | `\sens\fx_rate.csv` |
| CSA currency correspondence | `\sens\csa_currency.csv` |

### Let's Take a Look at the Trade file

In [5]:
pd.set_option('display.max_columns', 50)
trade_file_path = r'\trade\trade_data.csv'
def read_trade_data(path):
    return pd.read_csv(path, dtype = str,low_memory=False) 
trade_data = read_trade_data(input_path + trade_file_path)
print trade_data.head()

       trade_id                   netting_grp_id collateral_contract_id  \
0  303702592190  1478272006987275000000867689000         10000000088617   
1  303702592190  1478272006987275000000867689000         10000000088617   
2  303702592208   384950006987275000315034264000         10000000101574   
3  303702592208   384950006987275000315034264000         10000000101574   
4  303702592209   384950006987275000315034264000         10000000101574   

      notional    asset_class product_class         mtm trade_maturity_date  \
0   14931000.0  Interest Rate       RatesFX   116691.67          2017-08-24   
1   14931000.0  Interest Rate       RatesFX   116691.67          2017-08-24   
2  373275000.0  Interest Rate       RatesFX  -260707.03          2017-09-15   
3  373275000.0  Interest Rate       RatesFX  -260707.03          2017-09-15   
4  373275000.0  Interest Rate       RatesFX  -457758.03          2017-09-15   

  counterpartyOID transaction_id  ncmr_trade_id source_feed_name im_csa_id

### Filter the Trade File per Needs

The trade data can always be filtered per needs. For example, below we filter for only for CSA.

In [7]:
trade_data_sample = trade_data[trade_data.collateral_contract_id.isin(['10000000088617'])]
print trade_data_sample.head(10)

         trade_id                   netting_grp_id collateral_contract_id  \
0    303702592190  1478272006987275000000867689000         10000000088617   
1    303702592190  1478272006987275000000867689000         10000000088617   
54   303702592254  1478272006987275000000867689000         10000000088617   
55   303702592254  1478272006987275000000867689000         10000000088617   
202  303702593325  1478272006987275000000867689000         10000000088617   
203  303702593325  1478272006987275000000867689000         10000000088617   
236  303702593371  1478272006987275000000867689000         10000000088617   
237  303702593371  1478272006987275000000867689000         10000000088617   
322  303702594337  1478272006987275000000867689000         10000000088617   
323  303702594337  1478272006987275000000867689000         10000000088617   

        notional    asset_class product_class        mtm trade_maturity_date  \
0     14931000.0  Interest Rate       RatesFX  116691.67          2017-0

## Parameters

In [5]:
#Parameters for filtering trade files
regime_type = 'US-PR' #im regime type in trade population
im_direction = 'self' #im direction in trade population
run_single_csa = None #'10000000088617'

In [6]:
#Parameters that for customized usage
sens_folder = 'sens' 
model_folder = 'model'
TradeFileName = "trade_data_sample.csv" #"trade_data*.csv" for multiple runs
result_name_addon = ''

#SIMM internel Params
calculate_trade_level_marginal_im = False
return_standalone_in_marginal_im = True
im_calculation_direction = 'both_im' #both_im'/'reciprocal_im_only'/'self_im_only'
RunBaseline = False

## Run Calculator

In [7]:
sc.main(data_path = input_path
         ,calculate_trade_level_marginal_im = calculate_trade_level_marginal_im
         ,return_standalone_in_marginal_im = return_standalone_in_marginal_im
         ,im_calculation_direction = im_calculation_direction
         ,RunBaseline = RunBaseline
         ,regime_type = regime_type
         ,im_direction = im_direction
         ,sens_folder = sens_folder
         ,model_folder = model_folder
         ,TradeFileName = TradeFileName
         ,result_name_addon = result_name_addon
         ,run_single_csa = run_single_csa
         )

## Result

The result can be checked in the result folder `\result`.

In [12]:
result = pd.read_csv(input_path+r'\result\csa_im_results.csv',dtype = {"im_id":str})
result_detail = pd.read_csv(input_path+r'\result\csa_im_asset_level.csv',dtype = {"im_id":str})
result

,im_id,margin_type,im_value,asset_class,is_reciprocal,scenario_name
0,10000000065581,self,272931.746896,_Backtesting,NaN,NaN
1,10000000065581,reciprocal,272931.746896,_Backtesting,NaN,NaN


In [11]:
result_detail

,Unnamed: 0,Unnamed: 1,self,self_usd,im_ccy,reciprocal,reciprocal_usd,Asset,Level,scenario_name
0,10000000102298,RatesFX,1.541508e+07,1.541508e+07,USD,1.541508e+07,1.541508e+07,GIRR,Delta,_Backtesting
1,10000000101587,RatesFX,3.456492e+06,3.456492e+06,USD,3.456492e+06,3.456492e+06,GIRR,Delta,_Backtesting
2,10000000101574,RatesFX,2.529004e+07,2.529004e+07,USD,2.529004e+07,2.529004e+07,GIRR,Delta,_Backtesting
3,10000000065911,RatesFX,9.867923e+07,9.867923e+07,USD,9.867923e+07,9.867923e+07,GIRR,Delta,_Backtesting
4,10000000099816,RatesFX,1.831697e+06,2.364720e+06,GBP,1.831697e+06,2.364720e+06,GIRR,Delta,_Backtesting
5,10000000103002,RatesFX,1.467418e+07,1.894436e+07,GBP,1.467418e+07,1.894436e+07,GIRR,Delta,_Backtesting
6,10000000102821,RatesFX,1.477319e+07,1.477319e+07,USD,1.477319e+07,1.477319e+07,GIRR,Delta,_Backtesting
7,10000000065581,RatesFX,1.076990e+08,1.076990e+08,USD,1.076990e+08,1.076990e+08,GIRR,Delta,_Backtesting
8,10000000097706,RatesFX,1.418649e+07,1.418649e+07,USD,1.418649e+07,1.418649e+07,GIRR,Delta,_Backtesting
9,10000000102298,RatesFX,1.565554e+05,1.565554e+05,USD,1.565554e+05,1.565554e+05,FX,Delta,_Backtesting


## What can SIMM be used for?

We currently use SIMM to do a lot of model development and backtesting stuff, such as
* SIMM vs. CCP;
* SIMM vs. PnL backtesting;
* SIMM projection;
* ...